# **SMU Course Bidding Data Preprocessing**

<div style="background-color:#DFFFD6; padding:12px; border-radius:5px; border: 1px solid #228B22;">
    
  <h2 style="color:#006400;">✅ Looking to Implement This? ✅</h2>
  
  <p>🚀 Get started quickly by using <strong><a href="example_prediction.ipynb">example_prediction.ipynb</a></strong>.</p>
  
  <ul>
    <li>📌 **Pre-trained CatBoost model (`.cbm`) available for instant predictions.**</li>
    <li>🔧 Includes **step-by-step instructions** for making predictions.</li>
    <li>⚡ Works **out-of-the-box**—just load the model and start predicting!</li>
  </ul>

  <h3>🔗 📌 Next Steps:</h3>
  <p>👉 <a href="example_prediction.ipynb"><strong>Go to Example Prediction Notebook</strong></a></p>

</div>

### **Changes in V4**
- Replaced `BidderCount` with `Before Process Vacancy` due to future dependent results like `After Process Vacancy` which is not available at prediction time.
- Development of two models, one for `Median Bid Price` and `Min Bid Price`.
- Refined model input to make ingesting data for prediction easier. No label encoding done for `Term` or `Round`.

### **Objective**
This notebook performs the following steps:
1. **Data Cleaning** - Handle redundant columns and remove unwanted data.
2. **Feature Engineering** - Create derived features.
3. **Exploratory Data Analysis (EDA)** - Analyze key features and outlier cleaning.
4. **Save Processed Data** - Save the data into a csv that is useable for other ML models

### **Requirements**
- Python 3.x
- TensorFlow, Pandas, NumPy, Matplotlib, Seaborn, Sklearn

---


---

## **1. Setup**

In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import glob
import re
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from typing import Dict, List, Tuple, Optional, Union
import warnings
warnings.filterwarnings('ignore')


---

## **2. SMUBiddingTransformer Class**

### **SMU Bidding Data Feature Engineering Transformer**

#### **What This Code Does**
The `SMUBiddingTransformer` class is a comprehensive feature engineering pipeline designed specifically for Singapore Management University (SMU) course bidding data and optimized for CatBoost model training. It transforms raw tabular bidding data into machine learning-ready features while preserving categorical features where beneficial for CatBoost and creating embeddings only for high-cardinality features.

**Key Features:**
- **CatBoost-Optimized**: Preserves categorical features in their natural form for CatBoost's superior categorical handling while creating embeddings only for high-cardinality features (instructors, day combinations)
- **Feature Type Tracking**: Automatically categorizes features into categorical, numeric, and embedding types with dedicated getter methods for each
- **Smart Missing Value Handling**: Configurable approach to missing values - either let CatBoost handle them natively or create embeddings for missing data
- **High-Cardinality Embeddings**: Creates dense vector representations of instructor names (1000+ unique) and day-of-week combinations using TF-IDF + SVD
- **Course Code Intelligence**: Parses various course code formats including hyphenated codes ('COR-COMM175') and standard formats ('MGMT715')
- **Bidding Window Parsing**: Extracts round numbers (including 1A, 1B, 2A formats) and window numbers from complex bidding window strings
- **Categorical Preservation**: Keeps start_time, term, course_name, subject_area as categorical for CatBoost's target encoding
- **Sklearn-Compatible**: Standard transformer interface with `fit()`, `transform()`, and `fit_transform()` methods

#### **What Is Required**

**Input Data Format:**
The transformer expects a pandas DataFrame with these **required columns**:
- `course_code` (str): Course identifier (e.g., 'MGMT715', 'COR-COMM175')
- `course_name` (str): Full course name
- `acad_year_start` (int): Academic year start (e.g., 2025)
- `term` (str): Academic term ('1', '2', '3A', '3B')
- `start_time` (str): Class start time (e.g., '19:30', 'TBA') - preserved as categorical
- `day_of_week` (str): Days of week, can be comma-separated (e.g., 'Mon,Thu')
- `before_process_vacancy` (int): Number of available vacancies
- `bidding_window` (str): Bidding window descriptor (e.g., 'Round 1 Window 1', 'Incoming Freshmen Rnd 1 Win 4')
- `instructor` (str): Instructor names, can be comma-separated (e.g., 'JOHN DOE, JANE SMITH')

**Technical Dependencies:**
- Python packages: `pandas`, `numpy`, `sklearn` (TfidfVectorizer, TruncatedSVD)
- Standard libraries: `typing`, `warnings`, `re`

**Configuration Parameters:**
- `n_instructor_components` (int, default=50): Embedding dimensions for instructor names (expected 1000+ unique instructors)
- `n_day_components` (int, default=20): Embedding dimensions for day-of-week combinations (handles 7! combinations)
- `use_embeddings_for_missing` (bool, default=False): If True, missing values get embeddings. If False (recommended), missing values remain as None/NaN for CatBoost to handle natively

#### **Output Format**
The transformer produces a pandas DataFrame with engineered features organized into three categories:

**Categorical Features** (for CatBoost's `cat_features` parameter):
- `subject_area`, `catalogue_no`, `round`, `term`, `start_time`, `course_name`

**Numeric Features:**
- `window`, `before_process_vacancy`, `acad_year_start`

**Embedding Features:**
- `instructor_embed_0` through `instructor_embed_{n_instructor_components-1}` (50 by default)
- `day_embed_0` through `day_embed_{n_day_components-1}` (20 by default)

#### **Usage in Jupyter Notebook**

**Basic Usage:**
```python
from your_module import SMUBiddingTransformer

# Initialize transformer
transformer = SMUBiddingTransformer(
    n_instructor_components=50,     # Instructor embedding size
    n_day_components=20,            # Day-of-week embedding size  
    use_embeddings_for_missing=False # Let CatBoost handle missing values
)

# Fit on training data and transform
X_train = transformer.fit_transform(training_dataframe)

# Transform new data (after fitting)
X_test = transformer.transform(test_dataframe)

# Get feature lists for CatBoost
categorical_features = transformer.get_categorical_features()
numeric_features = transformer.get_numeric_features()
embedding_features = transformer.get_embedding_features()
```

**CatBoost Integration:**
```python
from catboost import CatBoostRegressor

# Initialize transformer and prepare data
transformer = SMUBiddingTransformer(use_embeddings_for_missing=False)
X_train = transformer.fit_transform(training_dataframe)
X_test = transformer.transform(test_dataframe)

# Use transformer's feature categorization
model = CatBoostRegressor(
    cat_features=transformer.get_categorical_features(),
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    one_hot_max_size=10  # CatBoost will use target encoding for larger categories
)

# Fit and predict
model.fit(X_train, y_train)
predictions = model.predict(X_test)
```

**Feature Inspection:**
```python
# Check what features were generated
print("Transformed shape:", X_train.shape)
print("\nCategorical features for CatBoost:")
print(transformer.get_categorical_features())
print(f"\nNumeric features ({len(transformer.get_numeric_features())}):")
print(transformer.get_numeric_features())
print(f"\nEmbedding features ({len(transformer.get_embedding_features())}):")
print(transformer.get_embedding_features()[:10])  # First 10
```

**Resume Capability:**
- **Stateful Transformer**: Once fitted, maintains vectorizers and SVD components for consistent transformations
- **Missing Value Strategy**: Configurable handling - either create embeddings for missing data or let CatBoost handle them natively (recommended)
- **Feature Consistency**: Maintains consistent categorical/numeric/embedding separation across different datasets
- **Validation**: Validates input columns and provides clear error messages for missing required fields

**Notes:**
- Optimized specifically for CatBoost's categorical feature handling capabilities
- High-cardinality features (instructors, day combinations) get embeddings while low-cardinality features remain categorical
- The `use_embeddings_for_missing=False` default lets CatBoost handle missing values with its built-in missing value support
- Feature type tracking enables easy CatBoost configuration without manual feature specification

In [2]:
class SMUBiddingTransformer:
    """
    A reusable transformer class for processing SMU course bidding data
    optimized for CatBoost model.
    
    This transformer preserves categorical features where beneficial for CatBoost
    while creating embeddings for high-cardinality features (instructors, day combinations).
    
    Expected input columns:
    - course_code: str (e.g. 'MGMT715', 'COR-COMM175')
    - course_name: str
    - acad_year_start: int
    - term: str ('1', '2', '3A', '3B')
    - start_time: str (e.g. '19:30', 'TBA') - preserved as categorical
    - day_of_week: str (can be multivalued, e.g. 'Mon,Thu')
    - before_process_vacancy: int
    - bidding_window: str (e.g. 'Round 1 Window 1', 'Incoming Freshmen Rnd 1 Win 4')
    - instructor: str (can be multivalued, e.g. 'JOHN DOE, JANE SMITH')
    """
    
    def __init__(self, 
                 n_instructor_components: int = 50, 
                 n_day_components: int = 20,
                 use_embeddings_for_missing: bool = False):
        """
        Initialize the transformer with embedding dimensions.
        
        Parameters:
        -----------
        n_instructor_components : int, default=50
            Number of dimensions for instructor embeddings (expected 1000+ unique instructors)
        n_day_components : int, default=20
            Number of dimensions for day_of_week embeddings (handles 7! combinations)
        use_embeddings_for_missing : bool, default=False
            If True, missing values get embeddings. If False, they remain as None/NaN
            for CatBoost to handle natively
        """
        self.n_instructor_components = n_instructor_components
        self.n_day_components = n_day_components
        self.use_embeddings_for_missing = use_embeddings_for_missing
        
        # Vectorizers for high-cardinality features only
        self.instructor_vectorizer = TfidfVectorizer(
            max_features=1000,
            token_pattern=r'\b\w+\b',
            ngram_range=(1, 2)  # Capture name variations
        )
        self.instructor_svd = TruncatedSVD(
            n_components=n_instructor_components, 
            random_state=42
        )
        
        # Day combination vectorizer
        self.day_vectorizer = TfidfVectorizer(
            max_features=150,
            token_pattern=r'\b\w+\b'
        )
        self.day_svd = TruncatedSVD(
            n_components=n_day_components,
            random_state=42
        )
        
        # Fitted flags
        self.is_fitted = False
        
        # Lists to track categorical features for CatBoost
        self.categorical_features = []
        self.numeric_features = []
        self.embedding_features = []
        
    def fit(self, df: pd.DataFrame) -> 'SMUBiddingTransformer':
        """
        Fit the transformer on training data.
        
        Parameters:
        -----------
        df : pandas.DataFrame
            Training dataframe with all required columns
        """
        # Validate required columns
        required_cols = [
            'course_code', 'course_name', 'acad_year_start', 'term',
            'start_time', 'day_of_week', 'before_process_vacancy',
            'bidding_window', 'instructor'
        ]
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Process instructors for embeddings (high cardinality)
        instructor_texts = self._process_instructor_for_embedding(df['instructor'])
        if self.use_embeddings_for_missing or any(text != '' for text in instructor_texts):
            instructor_tfidf = self.instructor_vectorizer.fit_transform(instructor_texts)
            self.instructor_svd.fit(instructor_tfidf)
        
        # Process day combinations for embeddings (high cardinality)
        day_texts = self._process_day_for_embedding(df['day_of_week'])
        if self.use_embeddings_for_missing or any(text != '' for text in day_texts):
            day_tfidf = self.day_vectorizer.fit_transform(day_texts)
            self.day_svd.fit(day_tfidf)
        
        self.is_fitted = True
        return self
    
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Transform the dataframe to CatBoost-ready format.
        
        Returns both the transformed dataframe and lists of categorical feature indices
        for CatBoost's cat_features parameter.
        """
        if not self.is_fitted:
            raise ValueError("Transformer must be fitted before transform. Call fit() first.")
        
        # Create a copy to avoid modifying original
        df_transformed = df.copy()
        
        # Reset feature tracking
        self.categorical_features = []
        self.numeric_features = []
        self.embedding_features = []
        all_features = []
        
        # 1. Extract course components (categorical + numeric)
        course_features = self._extract_course_features(df_transformed)
        all_features.append(course_features)
        
        # 2. Process bidding window (categorical + numeric)
        round_window_features = self._extract_round_window(df_transformed)
        all_features.append(round_window_features)
        
        # 3. Basic features (preserve categorical nature)
        basic_features = self._process_basic_features(df_transformed)
        all_features.append(basic_features)
        
        # 4. Create instructor embeddings (only for non-missing)
        instructor_embeddings = self._create_instructor_embeddings(df_transformed)
        if instructor_embeddings is not None:
            all_features.append(instructor_embeddings)
        
        # 5. Create day embeddings (only for non-missing)
        day_embeddings = self._create_day_embeddings(df_transformed)
        if day_embeddings is not None:
            all_features.append(day_embeddings)
        
        # Combine all features
        final_df = pd.concat(all_features, axis=1)
        
        return final_df
    
    def fit_transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """Fit the transformer and transform the data in one step."""
        self.fit(df)
        return self.transform(df)
    
    def get_categorical_features(self) -> List[str]:
        """Get list of categorical feature names for CatBoost."""
        return self.categorical_features.copy()
    
    def get_numeric_features(self) -> List[str]:
        """Get list of numeric feature names."""
        return self.numeric_features.copy()
    
    def get_embedding_features(self) -> List[str]:
        """Get list of embedding feature names."""
        return self.embedding_features.copy()
    
    def _extract_course_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Extract subject area and catalogue number from course code."""
        features = pd.DataFrame(index=df.index)
        
        def split_course_code(code):
            """Split course code into subject area and catalogue number."""
            if pd.isna(code):
                return None, None
            
            code = str(code).strip().upper()
            
            # Handle hyphenated codes like 'COR-COMM175'
            if '-' in code:
                parts = code.split('-')
                if len(parts) >= 2:
                    subject = '-'.join(parts[:-1])
                    # Extract number from last part
                    num_match = re.search(r'(\d+)', parts[-1])
                    if num_match:
                        return subject, int(num_match.group(1))
                    else:
                        # Try extracting from full last part
                        num_match = re.search(r'(\d+)', code)
                        if num_match:
                            return subject, int(num_match.group(1))
            
            # Standard format like 'MGMT715'
            match = re.match(r'([A-Z\-]+)(\d+)', code)
            if match:
                return match.group(1), int(match.group(2))
            
            return code, 0
        
        # Extract components
        splits = df['course_code'].apply(split_course_code)
        features['subject_area'] = splits.apply(lambda x: x[0] if x else None)
        features['catalogue_no'] = splits.apply(lambda x: x[1] if x else 0)
        
        # subject_area and catalogue_no are categorical for CatBoost
        self.categorical_features.extend(['subject_area', 'catalogue_no'])
        
        return features
    
    def _extract_round_window(self, df: pd.DataFrame) -> pd.DataFrame:
        """Extract round and window from bidding_window string."""
        features = pd.DataFrame(index=df.index)
        
        def parse_bidding_window(window_str):
            """Parse bidding window string into round and window number."""
            if pd.isna(window_str):
                return None, None
            
            window_str = str(window_str).strip()
            
            # Handle patterns from V4_01 notebook
            import re
            match = re.search(r'Round\s+(\d[A-C]?)\s+Window\s+(\d)', window_str, re.IGNORECASE)
            if match:
                return match.group(1), int(match.group(2))
            
            match = re.search(r'Rnd\s+(\d[A-C]?)\s+Win\s+(\d)', window_str, re.IGNORECASE)
            if match:
                return match.group(1), int(match.group(2))
            
            match = re.search(r'(\d[A-C]?)', window_str)
            if match:
                win_match = re.search(r'Window\s+(\d)|Win\s+(\d)', window_str, re.IGNORECASE)
                if win_match:
                    window_num = int(win_match.group(1) or win_match.group(2))
                    return match.group(1), window_num
                return match.group(1), 1
            
            return '1', 1
        
        # Extract round and window
        parsed = df['bidding_window'].apply(parse_bidding_window)
        features['round'] = parsed.apply(lambda x: x[0] if x else '1')
        features['window'] = parsed.apply(lambda x: x[1] if x else 1)
        
        # Round as categorical (preserves ordering like 1, 1A, 1B, 2, 2A)
        self.categorical_features.append('round')
        
        # Window as numeric
        self.numeric_features.append('window')
        
        return features
    
    def _process_basic_features(self, df: pd.DataFrame) -> pd.DataFrame:
            """Process basic features, preserving categorical nature where beneficial."""
            features = pd.DataFrame(index=df.index)
            
            # Numeric features
            features['before_process_vacancy'] = pd.to_numeric(
                df['before_process_vacancy'], errors='coerce'
            ).fillna(0)
            features['acad_year_start'] = pd.to_numeric(
                df['acad_year_start'], errors='coerce'
            ).fillna(2025)
            
            self.numeric_features.extend(['before_process_vacancy', 'acad_year_start'])
            
            # Categorical features
            features['term'] = df['term'].astype(str)
            features['start_time'] = df['start_time'].astype(str)
            features['course_name'] = df['course_name'].astype(str)
            features['section'] = df['section'].astype(str)  # Add section as categorical
            
            # Replace empty strings with None for proper CatBoost handling
            features.loc[features['start_time'].isin(['', 'nan']), 'start_time'] = None
            features.loc[features['course_name'].isin(['', 'nan']), 'course_name'] = None
            features.loc[features['section'].isin(['', 'nan']), 'section'] = None
            
            self.categorical_features.extend(['term', 'start_time', 'course_name', 'section'])
            
            return features
        
    def _process_instructor_for_embedding(self, instructor_series: pd.Series) -> List[str]:
        """Process instructor names for embedding - only non-missing values."""
        processed = []
        
        for instructor in instructor_series:
            if pd.isna(instructor) or str(instructor).strip() == '' or str(instructor).upper() == 'TBA':
                if self.use_embeddings_for_missing:
                    processed.append('MISSING_INSTRUCTOR')
                else:
                    processed.append('')  # Will result in zero embeddings
            else:
                # Combine multiple instructors
                names = [name.strip().upper() for name in str(instructor).split(',')]
                processed.append(' '.join(names))
        
        return processed
    
    def _process_day_for_embedding(self, day_series: pd.Series) -> List[str]:
        """Process day combinations for embedding - only non-missing values."""
        processed = []
        
        day_abbrev = {
            'MONDAY': 'MON', 'TUESDAY': 'TUE', 'WEDNESDAY': 'WED',
            'THURSDAY': 'THU', 'FRIDAY': 'FRI', 'SATURDAY': 'SAT', 'SUNDAY': 'SUN',
            'MON': 'MON', 'TUE': 'TUE', 'WED': 'WED', 'THU': 'THU',
            'FRI': 'FRI', 'SAT': 'SAT', 'SUN': 'SUN'
        }
        
        for days in day_series:
            if pd.isna(days) or str(days).strip() == '':
                if self.use_embeddings_for_missing:
                    processed.append('MISSING_DAY')
                else:
                    processed.append('')  # Will result in zero embeddings
            else:
                # Handle multiple days
                day_list = []
                for day in str(days).split(','):
                    day_upper = day.strip().upper()
                    day_list.append(day_abbrev.get(day_upper, day_upper))
                # Sort for consistency
                day_list.sort()
                processed.append('_'.join(day_list))
        
        return processed
    
    def _create_instructor_embeddings(self, df: pd.DataFrame) -> Optional[pd.DataFrame]:
        """Create instructor embeddings only for non-missing values."""
        instructor_texts = self._process_instructor_for_embedding(df['instructor'])
        
        # Check if we have any non-empty texts
        if not self.use_embeddings_for_missing and all(text == '' for text in instructor_texts):
            return None
        
        # Transform using fitted vectorizer
        instructor_tfidf = self.instructor_vectorizer.transform(instructor_texts)
        instructor_embeddings = self.instructor_svd.transform(instructor_tfidf)
        
        # Create dataframe with embedding columns
        embedding_cols = [f'instructor_embed_{i}' for i in range(self.n_instructor_components)]
        instructor_df = pd.DataFrame(
            instructor_embeddings,
            columns=embedding_cols,
            index=df.index
        )
        
        # Set embeddings to 0 for missing instructors if not using embeddings for missing
        if not self.use_embeddings_for_missing:
            mask = [text == '' for text in instructor_texts]
            instructor_df.loc[mask] = 0
        
        self.embedding_features.extend(embedding_cols)
        
        return instructor_df
    
    def _create_day_embeddings(self, df: pd.DataFrame) -> Optional[pd.DataFrame]:
        """Create day combination embeddings only for non-missing values."""
        day_texts = self._process_day_for_embedding(df['day_of_week'])
        
        # Check if we have any non-empty texts
        if not self.use_embeddings_for_missing and all(text == '' for text in day_texts):
            return None
        
        # Transform using fitted vectorizer
        day_tfidf = self.day_vectorizer.transform(day_texts)
        day_embeddings = self.day_svd.transform(day_tfidf)
        
        # Create dataframe with embedding columns
        embedding_cols = [f'day_embed_{i}' for i in range(self.n_day_components)]
        day_df = pd.DataFrame(
            day_embeddings,
            columns=embedding_cols,
            index=df.index
        )
        
        # Set embeddings to 0 for missing days if not using embeddings for missing
        if not self.use_embeddings_for_missing:
            mask = [text == '' for text in day_texts]
            day_df.loc[mask] = 0
        
        self.embedding_features.extend(embedding_cols)
        
        return day_df

    
    def get_feature_names(self) -> List[str]:
        """Get all feature names after transformation."""
        if not self.is_fitted:
            raise ValueError("Transformer must be fitted to get feature names.")
        
        return self.categorical_features + self.numeric_features + self.embedding_features

# # Example usage with CatBoost
# if __name__ == "__main__":
#     # Create sample data
#     sample_data = pd.DataFrame({
#         'course_code': ['MGMT715', 'COR-COMM175', 'ECON101', 'STAT201'],
#         'course_name': ['Strategic Management', 'Business Communication', 'Principles of Economics', 'Applied Statistics'],
#         'acad_year_start': [2025, 2025, 2025, 2025],
#         'term': ['1', '2', '3A', '1'],
#         'start_time': ['19:30', '14:00', 'TBA', '10:00'],
#         'day_of_week': ['Mon,Thu', 'Tue', '', 'Mon,Wed,Fri'],
#         'before_process_vacancy': [10, 5, 15, 8],
#         'bidding_window': ['Round 1 Window 1', 'Round 2A Window 3', 'Incoming Freshmen Rnd 1 Win 2', 'Round 1B Window 2'],
#         'instructor': ['JOHN DOE, JANE SMITH', 'ROBERT LEE', 'TBA', '']
#     })
    
#     # Initialize transformer
#     transformer = SMUBiddingTransformer(
#         n_instructor_components=30,
#         n_day_components=15,
#         use_embeddings_for_missing=False  # Let CatBoost handle missing values
#     )
    
#     # Fit and transform
#     X_train = transformer.fit_transform(sample_data)
    
#     print("Transformed shape:", X_train.shape)
#     print("\nCategorical features for CatBoost:")
#     print(transformer.get_categorical_features())
#     print("\nNumeric features:")
#     print(transformer.get_numeric_features()[:10])  # First 10
#     print("\nEmbedding features:")
#     print(transformer.get_embedding_features()[:10])  # First 10
    
#     # Example CatBoost integration
#     print("\n# CatBoost Usage Example:")
#     print("from catboost import CatBoostRegressor")
#     print("model = CatBoostRegressor(")
#     print(f"    cat_features={transformer.get_categorical_features()},")
#     print("    iterations=1000,")
#     print("    learning_rate=0.03,")
#     print("    depth=6")
#     print(")")

## **3. SMUDataMerger Class**

### **SMU Raw Data and BOSS Results Integration Pipeline**

#### **What This Code Does**
The `SMUDataMerger` class is a comprehensive data integration pipeline that combines SMU's raw course data with BOSS (Banner Online Self-Service) bidding results to create a unified dataset for machine learning analysis. It intelligently merges timing information from multiple data sources and creates course-level records suitable for the `SMUBiddingTransformer`.

**Key Features:**
- **Multi-Source Data Integration**: Combines Excel-based raw data (standalone + multiple sheets) with BOSS results from multiple Excel files
- **Intelligent Class Timing Aggregation**: Groups class sessions by `record_key` and consolidates timing information (days, times, venues, instructors) for courses with multiple sessions
- **Flexible BOSS Results Loading**: Automatically discovers and processes all Excel files in the `overallBossResults` folder with filename-based metadata extraction
- **Smart Course Matching**: Creates standardized course keys (`course_code_section_year_term`) for precise matching between raw data and BOSS results
- **Data Quality Handling**: Manages missing values, duplicate timings, and inconsistent formats across different data sources
- **Output Standardization**: Produces a clean dataset with columns mapped to `SMUBiddingTransformer` requirements
- **Comprehensive Logging**: Tracks merge statistics, missing data, and data quality issues throughout the process

#### **What Is Required**

**Input Data Structure:**
- **Raw Data Excel File** (`script_input/raw_data.xlsx`):
  - `standalone` sheet: Core course information with columns like `course_code`, `section`, `acad_year_start`, `term`, `record_key`
  - `multiple` sheet: Detailed class sessions with `type`, `day_of_week`, `start_time`, `venue`, `professor_name`, `record_key`
- **BOSS Results Folder** (`script_input/overallBossResults/`):
  - Multiple Excel files containing bidding results with columns: `Course Code`, `Section`, `Term`, `Before Process Vacancy`, `Bidding Window`, `Instructor`, `Median Bid`, `Min Bid`, `Vacancy`

**Technical Dependencies:**
- Python packages: `pandas`, `glob`, `os`, `re`, `datetime`, `collections.Counter`
- Excel file reading capabilities (openpyxl or xlrd)

**Directory Structure:**
- Input: `script_input/raw_data.xlsx` and `script_input/overallBossResults/*.xlsx`
- Output: `script_output/model_training/model_data_{timestamp}.csv`

**Configuration Parameters:**
- `raw_data_path` (str, default="script_input/raw_data.xlsx"): Path to raw data Excel file
- `boss_results_folder` (str, default="script_input/overallBossResults"): Folder containing BOSS results Excel files

#### **Output Format**
The merger produces a timestamped CSV file with columns mapped to `SMUBiddingTransformer` requirements:

**Primary Columns:**
- `course_code`, `course_name`, `acad_year_start`, `term`, `section`
- `start_time`, `day_of_week` (aggregated from multiple class sessions)
- `before_process_vacancy`, `bidding_window`, `instructor`
- `median_bid`, `min_bid`, `vacancy`, `grading_basis`

**Data Aggregation Logic:**
- **Days**: Comma-separated unique days (e.g., "Mon, Wed, Fri")
- **Start Time**: Most common start time across sessions
- **Venue**: Comma-separated venue list
- **Instructors**: Comma-separated instructor names

#### **Usage in Jupyter Notebook**

**Basic Usage:**
```python
from your_module import SMUDataMerger

# Initialize merger with default paths
merger = SMUDataMerger()

# Execute the complete merge process
final_dataset = merger.process_and_merge()

# Check results
print(f"Final dataset shape: {final_dataset.shape}")
print(f"Unique courses: {final_dataset['course_code'].nunique()}")
```

**Custom Configuration:**
```python
# Custom paths
merger = SMUDataMerger(
    raw_data_path="custom_path/raw_data.xlsx",
    boss_results_folder="custom_path/boss_results"
)

# Execute merge
final_dataset = merger.process_and_merge()
```

**Step-by-Step Processing:**
```python
# For debugging or custom processing
merger = SMUDataMerger()

# Step 1: Load raw data
standalone_df, class_df = merger.load_raw_data()

# Step 2: Process timing aggregation
timing_summary = merger.process_class_timings(class_df)

# Step 3: Load BOSS results
boss_df = merger.load_boss_results()

# Step 4: Perform merge
final_df, detailed_df = merger.merge_data(standalone_df, timing_summary, boss_df)
```

**Integration with SMUBiddingTransformer:**
```python
# Complete pipeline from raw data to ML features
merger = SMUDataMerger()
transformer = SMUBiddingTransformer()

# Step 1: Merge raw data sources
merged_data = merger.process_and_merge()

# Step 2: Transform to ML features
ml_features = transformer.fit_transform(merged_data)

# Ready for CatBoost training
print(f"ML-ready features: {ml_features.shape}")
```

**Resume Capability:**
- **Automatic Output Management**: Creates timestamped output files to avoid overwriting previous merges
- **Data Validation**: Validates merge quality and reports statistics on successful matches
- **Error Handling**: Gracefully handles missing files, corrupted data, and merge failures
- **Logging**: Comprehensive logging of merge statistics, missing data patterns, and data quality issues

**Notes:**
- Designed specifically for SMU's data structure with `record_key` linking between sheets
- Handles complex class scheduling where courses have multiple sessions (lectures, tutorials, labs)
- Inner join strategy ensures only courses with complete raw data + BOSS results are included
- Output format directly compatible with `SMUBiddingTransformer` input requirements
- Automatic timestamp-based file naming prevents accidental data overwriting

In [3]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import glob
import re
from datetime import datetime


class SMUDataMerger:
    """
    A class to merge SMU raw data with boss results data for bidding analysis.
    """
    
    def __init__(self, raw_data_path="script_input/raw_data.xlsx", 
                 boss_results_folder="script_input/overallBossResults"):
        self.raw_data_path = raw_data_path
        self.boss_results_folder = boss_results_folder
        self.output_folder = "script_output/model_training"
        
        # Create output directory if it doesn't exist
        os.makedirs(self.output_folder, exist_ok=True)
    
    def load_raw_data(self):
        """
        Load and process the raw_data.xlsx file with standalone and multiple sheets.
        """
        print(f"Loading raw data from {self.raw_data_path}")
        
        # Load both sheets
        standalone_df = pd.read_excel(self.raw_data_path, sheet_name='standalone')
        multiple_df = pd.read_excel(self.raw_data_path, sheet_name='multiple')
        
        print(f"Standalone sheet: {standalone_df.shape[0]} rows")
        print(f"Multiple sheet: {multiple_df.shape[0]} rows")
        
        # Filter multiple_df to only include CLASS entries (ignore EXAM)
        class_df = multiple_df[multiple_df['type'] == 'CLASS'].copy()
        print(f"Class entries in multiple sheet: {class_df.shape[0]} rows")
        
        return standalone_df, class_df

    def process_class_timings(self, class_df):
        """
        Process class timings by grouping by record_key and aggregating the timing information.
        """
        if class_df.empty:
            return pd.DataFrame(columns=['record_key', 'day_of_week', 'start_time'])
        
        def aggregate_days(days):
            # Remove NaN values and convert to set to remove duplicates
            valid_days = [day for day in days if pd.notna(day)]
            if not valid_days:
                return None
            unique_days = sorted(set(valid_days))
            return ', '.join(unique_days)
        
        def get_most_common_time(times):
            # Remove NaN values
            valid_times = [time for time in times if pd.notna(time)]
            if not valid_times:
                return None
            # Get most common time, or first occurrence if tie
            time_counts = Counter(valid_times)
            return time_counts.most_common(1)[0][0]
        
        # Group by record_key and aggregate
        timing_summary = class_df.groupby('record_key').agg({
            'day_of_week': aggregate_days,
            'start_time': get_most_common_time,
            'venue': lambda x: ', '.join([str(v) for v in x if pd.notna(v)]) if any(pd.notna(v) for v in x) else None,
            'professor_name': lambda x: ', '.join([str(p) for p in x if pd.notna(p)]) if any(pd.notna(p) for p in x) else None
        }).reset_index()
        
        return timing_summary

    def combine_raw_data(self, standalone_df, class_df):
        """
        Combine standalone and multiple (class) data into one flat dataset.
        """
        print("Combining standalone and class timing data...")
        
        # Process class timings first
        timing_summary = self.process_class_timings(class_df)
        
        # Merge standalone with timing summary
        if not timing_summary.empty:
            combined_df = pd.merge(
                standalone_df,
                timing_summary,
                on='record_key',
                how='left'
            )
        else:
            combined_df = standalone_df.copy()
            combined_df['day_of_week'] = None
            combined_df['start_time'] = None
            combined_df['venue'] = None
            combined_df['professor_name'] = None
        
        # Create boss-compatible term format: "2021-22 Term 1"
        def create_boss_term_format(row):
            if pd.notna(row['acad_year_start']) and pd.notna(row['acad_year_end']) and pd.notna(row['term']):
                year_start = int(row['acad_year_start'])
                year_end = str(int(row['acad_year_end']))[-2:]  # Last 2 digits
                term = str(row['term']).strip()
                if term.startswith('T'):
                    term = term[1:]  # Remove T prefix
                return f"{year_start}-{year_end} Term {term}"
            return None
        
        # Add boss_term_format column for matching
        combined_df['boss_term_format'] = combined_df.apply(create_boss_term_format, axis=1)
        
        print(f"Combined raw data shape: {combined_df.shape}")
        print(f"Sample boss term formats created: {combined_df['boss_term_format'].value_counts().head()}")
        
        return combined_df

    def standardize_term_format(self, term_str):
        """
        Convert term formats - remove T prefix if present.
        """
        if pd.isna(term_str):
            return None
        
        term_str = str(term_str).strip().upper()
        
        # If starts with 'T', remove it
        if term_str.startswith('T'):
            return term_str[1:]
        
        return term_str
    
    def clean_text_encoding(self, text):
        """
        Clean text encoding issues from web scraping.
        """
        if pd.isna(text):
            return text
        
        text = str(text)
        
        # Common encoding fixes
        replacements = {
            'â€"': '–',  # en dash
            'â€™': "'",  # apostrophe
            'â€œ': '"',  # left quote
            'â€': '"',   # right quote
            'Ã©': 'é',   # e acute
            'Ã¨': 'è',   # e grave
            'Ã ': 'à',   # a grave
            'Ã¢': 'â',   # a circumflex
            'Ã®': 'î',   # i circumflex
            'Ã´': 'ô',   # o circumflex
            'Ã»': 'û',   # u circumflex
            'Ã§': 'ç',   # c cedilla
            'â€¦': '...',  # ellipsis
            'â€‰': ' ',   # thin space
            'Â': '',      # non-breaking space artifact
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Remove any remaining non-ASCII characters that might cause issues
        # But keep common accented characters
        import unicodedata
        text = unicodedata.normalize('NFKD', text)
        
        return text.strip()

    def create_course_key(self, course_code, section, acad_year_start, term):
        """
        Create a standardized key for matching courses.
        Format: COURSECODE_SECTION_YEAR_TERM
        """
        if pd.isna(course_code) or pd.isna(section):
            return None
        
        # Clean course code and section
        course_code_clean = str(course_code).strip().upper()
        section_clean = str(section).strip().upper()
        
        # Handle academic year - ensure it's an integer
        if pd.notna(acad_year_start):
            acad_year = int(float(acad_year_start))
        else:
            return None
        
        # Standardize term format (remove T prefix if present)
        if pd.notna(term):
            term_clean = self.standardize_term_format(term)
        else:
            return None
        
        key = f"{course_code_clean}_{section_clean}_{acad_year}_{term_clean}"
        return key

    def load_boss_results(self):
        """
        Load and combine all Excel files from the overallBossResults folder.
        """
        print(f"Loading boss results from {self.boss_results_folder}")
        
        # Find all Excel files in the folder
        excel_files = glob.glob(os.path.join(self.boss_results_folder, "*.xlsx"))
        
        if not excel_files:
            print("No Excel files found in the boss results folder!")
            return pd.DataFrame()
        
        print(f"Found {len(excel_files)} Excel files")
        
        all_boss_data = []
        
        for file_path in excel_files:
            try:
                # Extract academic year and term from filename
                filename = os.path.basename(file_path)
                print(f"Processing file: {filename}")
                
                # Load the Excel file
                df = pd.read_excel(file_path)
                
                # Add source filename for tracking
                df['source_file'] = filename
                
                all_boss_data.append(df)
                
            except Exception as e:
                print(f"Error processing {file_path}: {str(e)}")
                continue
        
        if not all_boss_data:
            print("No valid data found in boss results files!")
            return pd.DataFrame()
        
        # Combine all dataframes
        combined_boss_df = pd.concat(all_boss_data, ignore_index=True)
        print(f"Combined boss results: {combined_boss_df.shape[0]} rows")
        
        return combined_boss_df

    def parse_term_info(self, term_str):
        """
        Parse term string like "2021-22 Term 2" to extract academic year start and term.
        """
        if pd.isna(term_str):
            return None, None
        
        try:
            # Pattern: "YYYY-YY Term X"
            match = re.match(r'(\d{4})-\d{2}\s+Term\s+(.+)', str(term_str).strip())
            if match:
                acad_year_start = int(match.group(1))
                term = match.group(2).strip()  # This will be "1", "2", "3A", "3B"
                return acad_year_start, term
        except Exception as e:
            print(f"Error parsing term '{term_str}': {e}")
        
        return None, None

    def merge_data(self, combined_raw_df, boss_df):
        """
        Merge the combined raw data with boss results data.
        """
        print("Starting data merge process...")
        
        # Filter out rows with missing terms to avoid duplicates
        print(f"\nRows before filtering missing terms: {len(combined_raw_df)}")
        combined_raw_df = combined_raw_df[combined_raw_df['term'].notna()].copy()
        print(f"Rows after filtering missing terms: {len(combined_raw_df)}")
        
        # Clean boss results - remove unnamed columns
        boss_columns_to_keep = ['Term', 'Session', 'Bidding Window', 'Course Code', 'Description', 
                            'Section', 'Vacancy', 'Opening Vacancy', 'Before Process Vacancy', 
                            'D.I.C.E', 'After Process Vacancy', 'Enrolled Students', 
                            'Median Bid', 'Min Bid', 'Instructor', 'School/Department', 'source_file']
        
        # Filter boss_df to only keep valid columns
        boss_df_clean = boss_df[[col for col in boss_columns_to_keep if col in boss_df.columns]].copy()
        
        # Parse boss results term information to extract year and term
        boss_df_clean[['boss_acad_year_start', 'boss_term']] = boss_df_clean['Term'].apply(
            lambda x: pd.Series(self.parse_term_info(x))
        )
        
        # Create course keys for raw data
        combined_raw_df['course_key'] = combined_raw_df.apply(
            lambda row: self.create_course_key(
                row['course_code'], 
                row['section'], 
                row['acad_year_start'], 
                row['term']  # This is the original term like T1, T2
            ), axis=1
        )
        
        # Create course keys for boss data
        boss_df_clean['course_key'] = boss_df_clean.apply(
            lambda row: self.create_course_key(
                row['Course Code'], 
                row['Section'], 
                row['boss_acad_year_start'], 
                row['boss_term']  # This should already be in format like "1", "2", "3A"
            ), axis=1
        )
        
        # Debug: Show sample keys
        print(f"\nRaw data course keys (first 5):")
        print(combined_raw_df[['course_code', 'section', 'term', 'course_key']].dropna().head())
        print(f"\nBoss data course keys (first 5):")
        print(boss_df_clean[['Course Code', 'Section', 'Term', 'boss_term', 'course_key']].dropna().head())
        
        print(f"\nRaw data valid keys: {combined_raw_df['course_key'].notna().sum()} out of {len(combined_raw_df)}")
        print(f"Boss data valid keys: {boss_df_clean['course_key'].notna().sum()} out of {len(boss_df_clean)}")
        
        # Find common keys
        raw_keys = set(combined_raw_df['course_key'].dropna())
        boss_keys = set(boss_df_clean['course_key'].dropna())
        common_keys = raw_keys.intersection(boss_keys)
        print(f"\nCommon keys found: {len(common_keys)}")
        
        if len(common_keys) == 0:
            print("\nNo matching keys found. Checking for mismatches...")
            print("Sample raw keys:", list(raw_keys)[:5])
            print("Sample boss keys:", list(boss_keys)[:5])
        
        # Perform the merge
        merged_df = pd.merge(
            combined_raw_df,
            boss_df_clean,
            on='course_key',
            how='inner',
            suffixes=('_raw', '_boss')
        )
        
        print(f"\nMerged data: {merged_df.shape[0]} rows")
        
        if merged_df.empty:
            print("No matching records found between raw data and boss results.")
            return pd.DataFrame(), merged_df
        
        # Create the final dataframe with only required columns
        final_df = pd.DataFrame()
        
        # Map columns to match SMUBiddingTransformer expected input
        column_mapping = {
            'course_code': 'course_code',
            'course_name': 'course_name',
            'acad_year_start': 'acad_year_start',
            'term': 'term',  # Original term format
            'start_time': 'start_time',
            'day_of_week': 'day_of_week',
            'before_process_vacancy': 'Before Process Vacancy',
            'bidding_window': 'Bidding Window',
            'instructor': 'professor_name',  # Prefer professor_name from class data
            'median_bid': 'Median Bid',
            'min_bid': 'Min Bid',
            'section': 'section'
            # REMOVED: 'vacancy' and 'grading_basis'
        }
        
        # If professor_name is empty, use Instructor from boss results
        for new_col, source_col in column_mapping.items():
            if source_col in merged_df.columns:
                final_df[new_col] = merged_df[source_col]
            else:
                print(f"Warning: Column {source_col} not found in merged data")
                final_df[new_col] = None
        
        # Special handling for instructor - use boss Instructor if professor_name is empty
        if 'Instructor' in merged_df.columns:
            mask = final_df['instructor'].isna() | (final_df['instructor'] == '')
            final_df.loc[mask, 'instructor'] = merged_df.loc[mask, 'Instructor']
        
        # Add course description from boss results if course_name is missing
        if 'Description' in merged_df.columns:
            mask = final_df['course_name'].isna() | (final_df['course_name'] == '')
            final_df.loc[mask, 'course_name'] = merged_df.loc[mask, 'Description']
        
        # Clean course names and instructor names for encoding issues
        final_df['course_name'] = final_df['course_name'].apply(self.clean_text_encoding)
        final_df['instructor'] = final_df['instructor'].apply(self.clean_text_encoding)
        
        print(f"\nFinal dataframe columns: {list(final_df.columns)}")
        print(f"Final dataframe shape: {final_df.shape}")
        
        return final_df, merged_df
    
    def process_and_merge(self):
        """
        Main method to execute the data merging process and save results.
        """
        try:
            # Step 1: Load raw data
            standalone_df, class_df = self.load_raw_data()
            
            # Step 2: Combine standalone and class data into one flat file
            combined_raw_df = self.combine_raw_data(standalone_df, class_df)
            
            # Step 3: Load boss results
            boss_df = self.load_boss_results()
            
            if boss_df.empty:
                print("No boss results data found. Exiting.")
                return None
            
            # Step 4: Merge the combined raw data with boss results
            final_df, detailed_df = self.merge_data(combined_raw_df, boss_df)
            
            if final_df.empty:
                print("No matching records found between raw data and boss results.")
                return None
            
            # Clean text encoding issues in course_name and instructor
            print("\nCleaning text encoding issues...")
            final_df['course_name'] = final_df['course_name'].apply(self.clean_text_encoding)
            final_df['instructor'] = final_df['instructor'].apply(self.clean_text_encoding)
            
            # Step 5: Save the results with timestamp
            timestamp = datetime.now().strftime("%d%m%y%H%M%S")
            
            # Save classification model data (all data)
            classification_filename = f"classification_model_data_{timestamp}.csv"
            classification_path = os.path.join(self.output_folder, classification_filename)
            final_df.to_csv(classification_path, index=False)
            
            print(f"\nClassification model data saved to: {classification_path}")
            print(f"Classification dataset shape: {final_df.shape}")
            
            # Create and save CatBoost model data (non-zero bids only)
            catboost_df = final_df[(final_df['median_bid'] > 0) & (final_df['min_bid'] > 0)].copy()
            catboost_filename = f"catboost_model_data_{timestamp}.csv"
            catboost_path = os.path.join(self.output_folder, catboost_filename)
            catboost_df.to_csv(catboost_path, index=False)
            
            print(f"\nCatBoost model data saved to: {catboost_path}")
            print(f"CatBoost dataset shape: {catboost_df.shape}")
            print(f"Removed {len(final_df) - len(catboost_df)} rows with zero bids")
            
            # Display summary statistics
            print(f"\nSummary Statistics (Classification Data):")
            print(f"- Total merged records: {final_df.shape[0]}")
            print(f"- Unique courses: {final_df['course_code'].nunique()}")
            print(f"- Unique sections: {final_df['section'].nunique()}")
            print(f"- Academic years covered: {final_df['acad_year_start'].min()} - {final_df['acad_year_start'].max()}")
            print(f"- Terms covered: {sorted(final_df['term'].unique())}")
            
            # Check for missing critical values
            critical_cols = ['course_code', 'section', 'before_process_vacancy', 'median_bid', 'min_bid']
            for col in critical_cols:
                if col in final_df.columns:
                    missing_count = final_df[col].isna().sum()
                    if missing_count > 0:
                        print(f"- Missing values in {col}: {missing_count}")
            
            # Return the CatBoost dataframe as the primary output
            return catboost_df
            
        except Exception as e:
            print(f"Error during merge process: {str(e)}")
            raise

In [4]:
# Extract data required for model
merger = SMUDataMerger()
result_df = merger.process_and_merge()

Loading raw data from script_input/raw_data.xlsx
Standalone sheet: 12973 rows
Multiple sheet: 19986 rows
Class entries in multiple sheet: 13082 rows
Combining standalone and class timing data...
Combined raw data shape: (12973, 26)
Sample boss term formats created: boss_term_format
2023-24 Term 2    1664
2023-24 Term 1    1659
2024-25 Term 1    1647
2021-22 Term 2    1631
2022-23 Term 1    1614
Name: count, dtype: int64
Loading boss results from script_input/overallBossResults
Found 14 Excel files
Processing file: 2021-22_T2.xlsx
Processing file: 2021-22_T3B.xlsx
Processing file: 2022-23_T1.xlsx
Processing file: 2022-23_T2.xlsx
Processing file: 2022-23_T3A.xlsx
Processing file: 2022-23_T3B.xlsx
Processing file: 2023-24_T1.xlsx
Processing file: 2023-24_T2.xlsx
Processing file: 2023-24_T3A.xlsx
Processing file: 2023-24_T3B.xlsx
Processing file: 2024-25_T1.xlsx
Processing file: 2024-25_T2.xlsx
Processing file: 2024-25_T3A.xlsx
Processing file: 2024-25_T3B.xlsx
Combined boss results: 12134